# Implementing logistic regression from scratch

The goal of this notebook is to implement your own logistic regression classifier. You will:

 * Extract features from Amazon product reviews.
 * Convert an SFrame into a NumPy array.
 * Implement the link function for logistic regression.
 * Write a function to compute the derivative of the log likelihood function with respect to a single coefficient.
 * Implement gradient ascent.
 * Given a set of coefficients, predict sentiments.
 * Compute classification accuracy for the logistic regression model.
 
Let's get started!

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
os.listdir()

['amazon_baby_subset.csv',
 'module-3-assignment-numpy-arrays.npz',
 'CLA03-NB01.ipynb',
 'amazon_baby_subset.sframe',
 '.ipynb_checkpoints',
 'amazon_baby_subset.gl',
 'c3w2_Logistic-regression_sklearn.ipynb',
 'important_words.json']

## Load review dataset
**1**. For this assignment, we will use a subset of the Amazon product review dataset. The subset was chosen to contain similar numbers of positive and negative reviews, as the original dataset consisted primarily of positive reviews.

Load the dataset into a data frame named **products**. 

In [3]:
products = pd.read_csv('amazon_baby_subset.csv')

**2**. Let us quickly explore more of this dataset.

In [4]:
products.head()

,name,review,rating,sentiment
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1
2,Nature's Lullabies Second Year Sticker Calendar,My daughter had her 1st baby over a year ago. ...,5,1
3,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,1
4,SoftPlay Peek-A-Boo Where's Elmo A Children's ...,Very cute interactive book! My son loves this ...,5,1


One column of this dataset is **sentiment**, corresponding to the class label with +1 indicating a review with positive sentiment and -1 indicating one with negative sentiment.

In [5]:
print(products['sentiment'][:10])

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: sentiment, dtype: int64


Let us quickly explore more of this dataset. The **name** column indicates the name of the product. Here we list the first 10 products in the dataset. We then count the number of positive and negative reviews.

In [6]:
products.head(10)['name']

0    Stop Pacifier Sucking without tears with Thumb...
1      Nature's Lullabies Second Year Sticker Calendar
2      Nature's Lullabies Second Year Sticker Calendar
3                          Lamaze Peekaboo, I Love You
4    SoftPlay Peek-A-Boo Where's Elmo A Children's ...
5                            Our Baby Girl Memory Book
6    Hunnt&reg; Falling Flowers and Birds Kids Nurs...
7    Blessed By Pope Benedict XVI Divine Mercy Full...
8    Cloth Diaper Pins Stainless Steel Traditional ...
9    Cloth Diaper Pins Stainless Steel Traditional ...
Name: name, dtype: object

After that, try counting the number of positive and negative reviews.

In [7]:
print(f"# of positive reviews = {sum(products['sentiment'] == 1)}")
print(f"# of negative reviews = {sum(products['sentiment'] == -1)}")

# of positive reviews = 26579
# of negative reviews = 26493


**Note:** For this assignment, we eliminated class imbalance by choosing 
a subset of the data with a similar number of positive and negative reviews. 

## Apply text cleaning on the review data

**3**. In this section, we will perform some simple feature cleaning using data frames. The last assignment used all words in building bag-of-words features, but here we limit ourselves to 193 words (for simplicity). We compiled a list of 193 most frequent words into the JSON file named **important_words.json**. Load the words into a list **important_words**.

Now, we will load these words from this JSON file:

In [8]:
import json
with open('important_words.json', 'r') as f: # Reads the list of most frequent words
    important_words = json.load(f)
important_words = [str(s) for s in important_words]

In [9]:
print(important_words)

['baby', 'one', 'great', 'love', 'use', 'would', 'like', 'easy', 'little', 'seat', 'old', 'well', 'get', 'also', 'really', 'son', 'time', 'bought', 'product', 'good', 'daughter', 'much', 'loves', 'stroller', 'put', 'months', 'car', 'still', 'back', 'used', 'recommend', 'first', 'even', 'perfect', 'nice', 'bag', 'two', 'using', 'got', 'fit', 'around', 'diaper', 'enough', 'month', 'price', 'go', 'could', 'soft', 'since', 'buy', 'room', 'works', 'made', 'child', 'keep', 'size', 'small', 'need', 'year', 'big', 'make', 'take', 'easily', 'think', 'crib', 'clean', 'way', 'quality', 'thing', 'better', 'without', 'set', 'new', 'every', 'cute', 'best', 'bottles', 'work', 'purchased', 'right', 'lot', 'side', 'happy', 'comfortable', 'toy', 'able', 'kids', 'bit', 'night', 'long', 'fits', 'see', 'us', 'another', 'play', 'day', 'money', 'monitor', 'tried', 'thought', 'never', 'item', 'hard', 'plastic', 'however', 'disappointed', 'reviews', 'something', 'going', 'pump', 'bottle', 'cup', 'waste', 'retu

**4**. Now, Let us perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Compute word counts (only for **important_words**)

We start with *Step 1* which can be done as follows:

* fill n/a values in the **review** column with empty strings. The n/a values indicate empty reviews. For instance, Pandas's the `fillna()` method lets you replace all N/A's in the **review** columns as follows:

In [10]:
products['review'].isnull().sum()

241

In [11]:
products = products.fillna({'review':''}) # fill in N/A's in the review column

In [12]:
products['review'].isnull().sum()

0

* Write a function **remove_punctuation** that takes a line of text and removes all punctuation from that text. The function should be analogous to the following Python code:

In [13]:
import re
import string

## method 1
# match everything that is not a lowercase a-z, uppercase A-Z, or digits 0-9, and replaces them with a space.
# def remove_punctuation(text):
#     text = str(text).lower() 
#     return re.sub(r"[^a-zA-Z0-9]", " ", str(text))


## methods 2
def remove_punctuation(text):
    translator = text.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    return text


* Apply the **remove_punctuation** function on every element of the **review** column and assign the result to the new column **review_clean**.   
    **Note**. Many data frame packages support **apply** operation for this type of task. Consult appropriate manuals.

In [14]:
products['review_clean'] = products['review'].apply(remove_punctuation)

In [15]:
products['review'].head()

0    All of my kids have cried non-stop when I trie...
1    We wanted to get something to keep track of ou...
2    My daughter had her 1st baby over a year ago. ...
3    One of baby's first and favorite books, and it...
4    Very cute interactive book! My son loves this ...
Name: review, dtype: object

In [16]:
products['review_clean'].head()

0    All of my kids have cried nonstop when I tried...
1    We wanted to get something to keep track of ou...
2    My daughter had her 1st baby over a year ago S...
3    One of babys first and favorite books and it i...
4    Very cute interactive book My son loves this b...
Name: review_clean, dtype: object

**5**. Now we proceed with *Step 2*. For each word in **important_words**, we compute a count for the number of times the word occurs in the review. We will store this count in a separate column (one for each word). The result of this feature processing is a single column for each word in **important_words** which keeps a count of the number of times the respective word occurs in the review text.


**Note:** There are several ways of doing this. One way is to create an anonymous function that counts the occurrence of a particular word and apply it to every element in the **review_clean** column. Repeat this step for every word in **important_words**. Your code should be analogous to the following:

In [17]:
for word in important_words:
    products[word] = products['review_clean'].apply(lambda x: x.split().count(word))

**6**. After #4 and #5,  the data frame **products** should contain one column for each of the 193 **important_words**. As an example, the column **perfect** contains a count of the number of times the word **perfect** occurs in each of the reviews.



In [18]:
products.head()

,name,review,rating,sentiment,review_clean,baby,one,great,love,use,...,seems,picture,completely,wish,buying,babies,won,tub,almost,either
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1,All of my kids have cried nonstop when I tried...,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1,We wanted to get something to keep track of ou...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Nature's Lullabies Second Year Sticker Calendar,My daughter had her 1st baby over a year ago. ...,5,1,My daughter had her 1st baby over a year ago S...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,1,One of babys first and favorite books and it i...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,SoftPlay Peek-A-Boo Where's Elmo A Children's ...,Very cute interactive book! My son loves this ...,5,1,Very cute interactive book My son loves this b...,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


**7**. Now, write some code to compute the number of product reviews that contain the word **perfect**.

**Hint**: 
* First create a column called `contains_perfect` which is set to 1 if the count of the word **perfect** (stored in column **perfect**) is >= 1.
* Sum the number of 1s in the column `contains_perfect`.

In [19]:
products['contains_perfect'] = products['perfect'].apply(lambda x: 1 if x >= 1 else 0)

In [20]:
sum(products['contains_perfect'])

2955

**Quiz Question**. How many reviews contain the word **perfect**?

**Answer**: 2955

## Convert data frame to multi-dimensional array
**8**. It is now time to convert our data frame to a multi-dimensional array. Look for a package that provides a highly optimized matrix operations. In the case of Python, NumPy is a good choice.

Write a function that extracts columns from a data frame and converts them into a multi-dimensional array. We plan to use them throughout the course, so make sure to get this function right.

The function should accept three parameters:

* **dataframe**: a data frame to be converted
* **features**: a list of string, containing the names of the columns that are used as features.
* **label**: a string, containing the name of the single column that is used as class labels.

The function should return two values:

* one 2D array for features
* one 1D array for class labels

The function should do the following:

* Prepend a new column **constant** to **dataframe** and fill it with 1's. This column takes account of the intercept term. Make sure that the **constant** column appears first in the data frame.

* Prepend a string 'constant' to the list **features**. Make sure the string 'constant' appears first in the list.

* Extract columns in **dataframe** whose names appear in the list **features**.

* Convert the extracted columns into a 2D array using a function in the data frame library. If you are using Pandas, you would use `.values` .

* Extract the single column in **dataframe** whose name corresponds to the string **label**.

* Convert the column into a 1D array.

* Return the 2D array and the 1D array.

Users of SFrame or Pandas would execute these steps as follows:

In [21]:
def get_numpy_data(df, features, label):
    df['constant'] = 1
    features = ['constant'] + features
    features_2d = df[features].values
    label_1d = df[label].values
    
    return features_2d, label_1d

**9**. Using the function written in #8, extract two arrays **feature_matrix** and **sentiment**. The 2D array **feature_matrix** would contain the content of the columns given by the list **important_words**. The 1D array **sentiment** would contain the content of the column **sentiment**.



In [22]:
feature_matrix, sentiment = get_numpy_data(products, important_words, 'sentiment')

In [23]:
feature_matrix

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [24]:
sentiment

array([ 1,  1,  1, ..., -1, -1, -1])

**Quiz Question**: How many features are there in the **feature_matrix**?  

**Answer**: 194

In [25]:
feature_matrix.shape

(53072, 194)

**Quiz Question**: Assuming that the intercept is present, how does the number of features in **feature_matrix** relate to the number of features in the logistic regression model?  Let x = [number of features in feature_matrix] and y = [number of features in logistic regression model].     

**Answer**: y = x

## Estimating conditional probability with link function
**10**. Recall from lecture that the link function is given by:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))},
$$

where the feature vector $h(\mathbf{x}_i)$ represents the word counts of **important_words** in the review  $\mathbf{x}_i$.  

Write a function named **predict_probability** that implements the link function.

* Take two parameters: **feature_matrix** and **coefficients**.
* First compute the dot product of **feature_matrix** and **coefficients**.
* Then compute the link function $P(y=+1|\mathbf{x},\mathbf{w})$.
* Return the predictions given by the link function.

Your code should be analogous to the following Python function:

In [26]:
def predict_probability(feature_matrix, coefficients):
    '''
    produces probablistic estimate for P(y_i = +1 | x_i, w).
    estimate ranges between 0 and 1.
    '''
    # Take dot product of feature_matrix and coefficients
    score = np.dot(feature_matrix, coefficients)
    
    # Compute P(y_i = +1 | x_i, w) using the link function
    predictions = 1 / (1 + np.exp(-score))
    
    return predictions

**Aside**. How the link function works with matrix algebra

Since the word counts are stored as columns in **feature_matrix**, each $i$-th row of the matrix corresponds to the feature vector $h(\mathbf{x}_i)$:
$$
[\text{feature_matrix}] =
\left[
\begin{array}{c}
h(\mathbf{x}_1)^T \\
h(\mathbf{x}_2)^T \\
\vdots \\
h(\mathbf{x}_N)^T
\end{array}
\right] =
\left[
\begin{array}{cccc}
h_0(\mathbf{x}_1) & h_1(\mathbf{x}_1) & \cdots & h_D(\mathbf{x}_1) \\
h_0(\mathbf{x}_2) & h_1(\mathbf{x}_2) & \cdots & h_D(\mathbf{x}_2) \\
\vdots & \vdots & \ddots & \vdots \\
h_0(\mathbf{x}_N) & h_1(\mathbf{x}_N) & \cdots & h_D(\mathbf{x}_N)
\end{array}
\right]
$$

By the rules of matrix multiplication, the score vector containing elements $\mathbf{w}^T h(\mathbf{x}_i)$ is obtained by multiplying **feature_matrix** and the coefficient vector $\mathbf{w}$.
$$
[\text{score}] =
[\text{feature_matrix}]\mathbf{w} =
\left[
\begin{array}{c}
h(\mathbf{x}_1)^T \\
h(\mathbf{x}_2)^T \\
\vdots \\
h(\mathbf{x}_N)^T
\end{array}
\right]
\mathbf{w}
= \left[
\begin{array}{c}
h(\mathbf{x}_1)^T\mathbf{w} \\
h(\mathbf{x}_2)^T\mathbf{w} \\
\vdots \\
h(\mathbf{x}_N)^T\mathbf{w}
\end{array}
\right]
= \left[
\begin{array}{c}
\mathbf{w}^T h(\mathbf{x}_1) \\
\mathbf{w}^T h(\mathbf{x}_2) \\
\vdots \\
\mathbf{w}^T h(\mathbf{x}_N)
\end{array}
\right]
$$

**Checkpoint**

Just to make sure you are on the right track, we have provided a few examples. If your `predict_probability` function is implemented correctly, then the outputs will match:

In [27]:
dummy_feature_matrix = np.array([[1.,2.,3.], [1.,-1.,-1]])
dummy_coefficients = np.array([1., 3., -1.])

correct_scores      = np.array( [ 1.*1. + 2.*3. + 3.*(-1.),          1.*1. + (-1.)*3. + (-1.)*(-1.) ] )
correct_predictions = np.array( [ 1./(1+np.exp(-correct_scores[0])), 1./(1+np.exp(-correct_scores[1])) ] )

print('The following outputs must match ')
print('------------------------------------------------')
print('correct_predictions           =', correct_predictions)
print('output of predict_probability =', predict_probability(dummy_feature_matrix, dummy_coefficients))

The following outputs must match 
------------------------------------------------
correct_predictions           = [0.98201379 0.26894142]
output of predict_probability = [0.98201379 0.26894142]


## Compute derivative of log likelihood with respect to a single coefficient

**11**. Recall from lecture:
$$
\frac{\partial\ell}{\partial w_j} = \sum_{i=1}^N h_j(\mathbf{x}_i)\left(\mathbf{1}[y_i = +1] - P(y_i = +1 | \mathbf{x}_i, \mathbf{w})\right)
$$

We will now write a function that computes the derivative of log likelihood with respect to a single coefficient $w_j$. The function accepts two arguments:
* `errors` vector containing $\mathbf{1}[y_i = +1] - P(y_i = +1 | \mathbf{x}_i, \mathbf{w})$ for all $i$.
* `feature` vector containing $h_j(\mathbf{x}_i)$  for all $i$. 

This corresponds to the j-th column of **feature_matrix**.

The function should do the following:

* Take two parameters **errors** and **feature**.
* Compute the dot product of **errors** and **feature**.
* Return the dot product. This is the derivative with respect to a single coefficient $w_j$.

In [28]:
def feature_derivative(errors, feature):     
    # Compute the dot product of errors and feature
    derivative = np.dot(errors, feature)
    
    return derivative

**12**. In the main lecture, our focus was on the likelihood.  In the advanced optional video, however, we introduced a transformation of this likelihood---called the log likelihood---that simplifies the derivation of the gradient and is more numerically stable.  Due to its numerical stability, we will use the log likelihood instead of the likelihood to assess the algorithm.

The log likelihood is computed using the following formula (see the advanced optional video if you are curious about the derivation of this equation):

$$\ell\ell(\mathbf{w}) = \sum_{i=1}^N \Big( (\mathbf{1}[y_i = +1] - 1)\mathbf{w}^T h(\mathbf{x}_i) - \ln\left(1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))\right) \Big) $$

Write a function **compute_log_likelihood** that implements the equation.

In [29]:
def compute_log_likelihood(feature_matrix, sentiment, coefficients):
    indicator = (sentiment == 1)
    scores = np.dot(feature_matrix, coefficients)
    ll = np.sum((indicator - 1) * scores - np.log(1 + np.exp(-scores)))
    
    return ll

**Checkpoint**

Just to make sure we are on the same page, run the following code block and check that the outputs match.

In [30]:
dummy_feature_matrix = np.array([[1.,2.,3.], [1.,-1.,-1]])
dummy_coefficients = np.array([1., 3., -1.])
dummy_sentiment = np.array([-1, 1])

correct_indicators  = np.array( [ -1==+1,                                       1==+1 ] )
correct_scores      = np.array( [ 1.*1. + 2.*3. + 3.*(-1.),                     1.*1. + (-1.)*3. + (-1.)*(-1.) ] )
correct_first_term  = np.array( [ (correct_indicators[0]-1)*correct_scores[0],  (correct_indicators[1]-1)*correct_scores[1] ] )
correct_second_term = np.array( [ np.log(1. + np.exp(-correct_scores[0])),      np.log(1. + np.exp(-correct_scores[1])) ] )

correct_ll          =      sum( [ correct_first_term[0]-correct_second_term[0], correct_first_term[1]-correct_second_term[1] ] ) 

print('The following outputs must match ')
print('------------------------------------------------')
print('correct_log_likelihood           =', correct_ll)
print('output of compute_log_likelihood =', compute_log_likelihood(dummy_feature_matrix, dummy_sentiment, dummy_coefficients))

The following outputs must match 
------------------------------------------------
correct_log_likelihood           = -5.331411615436032
output of compute_log_likelihood = -5.331411615436032


## Taking gradient steps
**13**. Now we are ready to implement our own logistic regression. All we have to do is to write a gradient ascent function that takes gradient steps towards the optimum. 

Write a function **logistic_regression** to fit a logistic regression model using gradient ascent.

The function accepts the following parameters:

* **feature_matrix**: 2D array of features
* **sentiment**: 1D array of class labels
* **initial_coefficients**: 1D array containing initial values of coefficients
* **step_size**: a parameter controlling the size of the gradient steps
* **max_iter**: number of iterations to run gradient ascent

The function returns the last set of coefficients after performing gradient ascent.

The function carries out the following steps:

1. Initialize vector **coefficients** to **initial_coefficients**.
2. Predict the class probability $P(yi=+1|\mathbf{x_i},\mathbf{w})$ using your **predict_probability** function and save it to variable **predictions**.
3. Compute indicator value for $(yi=+1)$ by comparing **sentiment** against +1. Save it to variable indicator.
4. Compute the errors as difference between **indicator** and **predictions**. Save the errors to variable **errors**.
5. For each j-th coefficient, compute the per-coefficient derivative by calling **feature_derivative** with the j-th column of **feature_matrix**. Then increment the j-th coefficient by (step_size*derivative).
6. Once in a while, insert code to print out the log likelihood.
7. Repeat steps 2-6 for **max_iter** times.

At the end of day, your code should be analogous to the following Python function (with blanks filled in):

In [31]:
from math import sqrt

def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter):
    coefficients = initial_coefficients
    for itr in range(max_iter):
        # Predict P(y_i = +1|x_1,w) using your predict_probability() function
        predictions = predict_probability(feature_matrix, coefficients)
        
        # Compute indicator value for (y_i = +1)
        indicator = (sentiment == +1)
        
        # Compute the errors as indicator - predictions
        errors = indicator - predictions
        
        for j in range(len(coefficients)): # loop over each coefficient
            # Recall that feature_matrix[:,j] is the feature column associated with coefficients[j]
            # compute the derivative for coefficients[j]. Save it in a variable called derivative
            derivative = feature_derivative(errors, feature_matrix[:, j])
            
            # add the step size times the derivative to the current coefficient
            coefficients[j] += step_size * derivative
            
        # Checking whether log likelihood is increasing
        if itr <=15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            ll = compute_log_likelihood(feature_matrix, sentiment, coefficients)
            print(f"iteration {itr:>{int(np.ceil(np.log10(max_iter)))}}: log likelihood of observed labels = {ll:.8f}")
            
    return coefficients

**14**. Now, let us run the logistic regression solver with the parameters below:

* **feature_matrix** = **feature_matrix** extracted in #9
* **sentiment** = **sentiment** extracted in #9
* **initial_coefficients** = a 194-dimensional vector filled with zeros
* **step_size** = 1e-7
* **max_iter** = 301

Save the returned coefficients to variable **coefficients**.

In [32]:
coefficients = logistic_regression(feature_matrix, 
                                   sentiment, 
                                   initial_coefficients=np.zeros(194), 
                                   step_size=1e-7, 
                                   max_iter=301)

iteration   0: log likelihood of observed labels = -36780.91768478
iteration   1: log likelihood of observed labels = -36775.13434712
iteration   2: log likelihood of observed labels = -36769.35713564
iteration   3: log likelihood of observed labels = -36763.58603240
iteration   4: log likelihood of observed labels = -36757.82101962
iteration   5: log likelihood of observed labels = -36752.06207964
iteration   6: log likelihood of observed labels = -36746.30919497
iteration   7: log likelihood of observed labels = -36740.56234821
iteration   8: log likelihood of observed labels = -36734.82152213
iteration   9: log likelihood of observed labels = -36729.08669961
iteration  10: log likelihood of observed labels = -36723.35786366
iteration  11: log likelihood of observed labels = -36717.63499744
iteration  12: log likelihood of observed labels = -36711.91808422
iteration  13: log likelihood of observed labels = -36706.20710739
iteration  14: log likelihood of observed labels = -36700.5020

**Quiz Question:** As each iteration of gradient ascent passes, does the log likelihood increase or decrease? 

**Answer**: It increases.

## Predicting sentiments
**15**. Recall from lecture that class predictions for a data point $\mathbf{x}$ can be computed from the coefficients $\mathbf{w}$ using the following formula:
$$
\hat{y}_i = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{x}_i^T\mathbf{w} > 0 \\
      -1 & \mathbf{x}_i^T\mathbf{w} \leq 0 \\
\end{array} 
\right.
$$

Now, we will write some code to compute class predictions. We will do this in two steps:
* **Step 1**: First compute the **scores** using **feature_matrix** and **coefficients** using a dot product.
* **Step 2**: Using the formula above, compute the class predictions from the scores.

Step 1 can be implemented as follows:

In [33]:
# Compute the scores as a dot product between feature_matrix and coefficients.
scores = np.dot(feature_matrix, coefficients)

Now, complete the following code block for **Step 2** to compute the class predictions using the **scores** obtained above:

In [34]:
y_hat = np.where(scores > 0 , +1, -1)
print(sum(y_hat == +1))

25126


**Quiz Question:** How many reviews were predicted to have positive sentiment?

**Answer**: 25126

## Measuring accuracy

**16**. We will now measure the classification accuracy of the model. Recall from the lecture that the classification accuracy can be computed as follows:

$$
\mbox{accuracy} = \frac{\mbox{# correctly classified data points}}{\mbox{# total data points}}
$$

Complete the following code block to compute the accuracy of the model.

In [35]:
num_mistakes = sum(y_hat != sentiment)
accuracy = sum(y_hat == sentiment) / len(products)

In [36]:
print("-----------------------------------------------------")
print('# Reviews   correctly classified =', len(products) - num_mistakes)
print('# Reviews incorrectly classified =', num_mistakes)
print('# Reviews total                  =', len(products))
print("-----------------------------------------------------")
print('Accuracy = %.2f' % accuracy)

-----------------------------------------------------
# Reviews   correctly classified = 39903
# Reviews incorrectly classified = 13169
# Reviews total                  = 53072
-----------------------------------------------------
Accuracy = 0.75


**Quiz Question**: What is the accuracy of the model on predictions made above? (round to 2 digits of accuracy)

**Answer**: 0.75

## Which words contribute most to positive & negative sentiments?
**17**. Recall that in the earlier assignment, we were able to compute the "**most positive words**". These are words that correspond most strongly with positive reviews. In order to do this, we will first do the following:
* Treat each coefficient as a tuple, i.e. (**word**, **coefficient_value**).
* Sort all the (**word**, **coefficient_value**) tuples by **coefficient_value** in descending order.

In [37]:
# exclude intercept
word_coef_tuples = [(w, c) for w, c in zip(important_words, coefficients[1:])]
word_coef_tuples = sorted(word_coef_tuples, key=lambda x: x[1], reverse=True)

Now, **word_coefficient_tuples** contains a sorted list of (**word**, **coefficient_value**) tuples. The first 10 elements in this list correspond to the words that are most positive.

### Ten "most positive" words

**18**. Compute the 10 words that have the most positive coefficient values. These words are associated with positive sentiment.

In [38]:
word_coef_tuples[:10]

[('great', 0.0665460841704577),
 ('love', 0.06589076292212327),
 ('easy', 0.0647945868025784),
 ('little', 0.04543562630842137),
 ('loves', 0.04497640139490604),
 ('well', 0.030135001092107084),
 ('perfect', 0.029739937104968462),
 ('old', 0.020077541034775385),
 ('nice', 0.018408707995268992),
 ('daughter', 0.017703199905701694)]

**Quiz Question:** Which word is **not** present in the top 10 "most positive" words?

- love
- easy
- great
- perfect
- cheap

**Answer**: cheap

### Ten "most negative" words

Next, we repeat this exercise on the 10 most negative words.  That is, we compute the 10 words that have the most negative coefficient values. These words are associated with negative sentiment.

In [39]:
word_coef_tuples[::-1][:10]

[('would', -0.05386014844520314),
 ('product', -0.041511033392108904),
 ('money', -0.038982037286487116),
 ('work', -0.03306951529475273),
 ('even', -0.030051249236035804),
 ('disappointed', -0.028978976142317068),
 ('get', -0.02871155298019258),
 ('back', -0.027742697230661334),
 ('return', -0.026592778462247283),
 ('monitor', -0.024482100545891717)]

**Quiz Question:** Which word is **not** present in the top 10 "most negative" words?

- need
- work
- disappointed
- even
- return

**Answer**: need